In [70]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import sqlite3

con = sqlite3.connect('/content/gdrive/My Drive/study/bermuda/schools_postsv4.db')

In [0]:
data = []
with con:    
    cur = con.cursor()    
    cur.execute("SELECT * FROM schools")
    rows = cur.fetchall()
 
    for row in rows:
        data.append(row)

In [73]:
data[0]

(0,
 'schoolhate32',
 'Админ ,опубликуй пожалуйста)\n\nЗдравствуйте! Помогите пожалуйста найти бигля. Зовут Тори, потерялась в районе ул. Генерала Ивлиева в 7 утра, в гаражах. Боится всех, вряд ли к кому то подойдёт, но возможно подойдёт за еду. Есть только клеймо, адресник потеряла, новый только делается.\n89867283833 Анна\n89200544771 Александр\n\n#дд_пропала_собака\n______________________\nНадеюсь найдётся❗',
 2,
 0,
 0,
 0,
 0,
 1206,
 3,
 25,
 0,
 28,
 '2020-01-15')

Русская лемматизация на базе spacy какая-то грустная. Возьмем первоначальный текст, lowercase, откинем стоп-слова nltk


In [74]:
! pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
import re

In [0]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

In [77]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [78]:
! pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [0]:
from gensim.models import CoherenceModel

In [0]:
import gensim

In [0]:
'''
все школы
'''

cur_data = {}
for i in data:
    ind = i[0]
    sent = i[2].lower()
    tokens = [morph.parse(tok)[0].normal_form for tok in tokenizer.tokenize(sent) if tok.isalpha() and len(tok) > 2 and tok not in stop_words
              and bool(re.match(r'[а-я]', tok)) and morph.parse(tok)[0].tag.POS not in ['NPRO', "NUMR", "CONJ", "INTJ", "PRCL", "PREP"]]
    if tokens != []:
        cur_data[ind] = tokens  

In [105]:
len(data)

19572

In [106]:
len(cur_data)

18085

In [0]:
data_words = list(cur_data.values())

In [0]:
bigram = gensim.models.Phrases(data_words, min_count=100, threshold=500)

In [0]:
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [0]:
composed = {}
for k, v in cur_data.items():
    composed[k] = bigram_mod[v]

In [0]:
id2word = gensim.corpora.Dictionary(composed.values())

In [0]:
corpus = [id2word.doc2bow(text) for text in composed.values()]

In [113]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, #подбор!
                                           random_state=42,
                                           update_every=1,
                                           chunksize=128, 
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

Streaming output truncated to the last 5000 lines.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

In [114]:
lda_model.print_topics()

[(0,
  '0.019*"класс" + 0.017*"школа" + 0.015*"хотеть" + 0.012*"любить" + 0.012*"весь" + 0.012*"человек" + 0.012*"очень" + 0.011*"такой" + 0.010*"быть" + 0.009*"сегодня"'),
 (1,
  '0.011*"урок" + 0.009*"аня" + 0.008*"должный" + 0.006*"интересный" + 0.006*"шарага" + 0.005*"нужно" + 0.005*"ждать" + 0.005*"внимание" + 0.005*"новость" + 0.005*"егэ"'),
 (2,
  '0.020*"свой" + 0.019*"друг" + 0.014*"мочь" + 0.014*"идти" + 0.014*"красивый" + 0.011*"добрый" + 0.010*"твой" + 0.010*"понять" + 0.009*"мой" + 0.008*"каждый"')]

In [115]:

coherence_model_lda = CoherenceModel(model=lda_model, texts= composed.values(), dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
coherence_lda

0.42739564338855085

In [0]:
perplexity = lda_model.log_perplexity(corpus)

In [68]:
perplexity

-8.739957390454336

In [69]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.278134 -0.019805       1        1  60.641766
2     -0.163963 -0.156044       2        1  22.627308
1     -0.114171  0.175849       3        1  16.730927, topic_info=     Category         Freq      Term        Total  loglift  logprob
60    Default   628.000000      свой   628.000000  30.0000  30.0000
89    Default   598.000000      друг   598.000000  29.0000  29.0000
100   Default  1558.000000     класс  1558.000000  28.0000  28.0000
152   Default  1401.000000     школа  1401.000000  27.0000  27.0000
284   Default   422.000000      мочь   422.000000  26.0000  26.0000
...       ...          ...       ...          ...      ...      ...
5833   Topic3    70.071602    машина    70.990906   1.7749  -5.7830
1610   Topic3   132.631348    шарага   138.242905   1.7465  -5.1449
2592   Topic3    76.144028  закрытый    77.623306   1.7687  -5.6999
1840   Topic3    78.369499  передать    81.188751   1.7526  -5.6711
2754   Topic3    81.663887   колледж   166.826004   1.0736  -5.6299

[129 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
76        1  0.004780     аня
76        3  0.994192     аня
79        1  0.999086    быть
51        1  0.999394     ваш
154       1  0.999288    весь
...     ...       ...     ...
1610      2  0.014467  шарага
1610      3  0.962075  шарага
152       1  0.999795   школа
184       3  0.991449    этаж
756       3  0.992927    язык

[120 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

# LDA Mallet

In [116]:
! wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

--2020-02-17 17:52:34--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip.1’

mallet-2.0.8.zip.1  100%[===================>]  15.43M  10.1MB/s    in 1.5s    

2020-02-17 17:52:36 (10.1 MB/s) - ‘mallet-2.0.8.zip.1’ saved [16184794/16184794]



In [117]:
! unzip mallet-2.0.8.zip

Archive:  mallet-2.0.8.zip
replace mallet-2.0.8/bin/classifier2info? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
mallet_path = 'mallet-2.0.8/bin/mallet'

In [119]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=3, id2word=id2word)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [120]:
ldamallet.show_topics(formatted=False)

[(0,
  [('школа', 0.047513280820663126),
   ('анон', 0.028920131892287964),
   ('ребята', 0.023768089393661842),
   ('наш', 0.01779172009525554),
   ('класс', 0.01726506686206265),
   ('сегодня', 0.014746290529400989),
   ('знать', 0.014723392562740429),
   ('пост', 0.009594248030774867),
   ('дискотека', 0.009548452097453745),
   ('учитель', 0.009113390730903096)]),
 (1,
  [('весь', 0.02154649561051831),
   ('человек', 0.017701844199099488),
   ('быть', 0.015728119410349718),
   ('который', 0.015316926746026851),
   ('свой', 0.015173009313513846),
   ('писать', 0.014679578116326406),
   ('ваш', 0.01128723863566274),
   ('один', 0.01122555973601431),
   ('друг', 0.011040523037069018),
   ('мочь', 0.008840642282941673)]),
 (2,
  [('хотеть', 0.02268253604943923),
   ('очень', 0.020668345159075303),
   ('класс', 0.019890135042343784),
   ('самый', 0.018448157473105974),
   ('такой', 0.017509727626459144),
   ('какой', 0.01586175326161593),
   ('любить', 0.01577019913023575),
   ('хороший'

In [121]:
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=composed.values(), dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
coherence_ldamallet

0.5107163171050138